In [1]:
!pip install -q openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [2]:
import openai
import os

In [33]:
openai.api_key = os.getenv('API_KEY')


In [121]:
messages = [
    {"role": "system", "content": "Suppose you are an analyst who needs to take a user review of an application as input and extract all the app problems, and the corresponding user actions that the user takes to encounter the app problem. \nAn app problem is defined as a single problem faced when interacting with an application. A user action is defined as a single interaction with the application. \nShow the outputs in the form of python list of dictionaries.\nThe list will contain dictionaries, each dictionary will have app problem as the key and a list of user actions in sequential order as values. It is based on the following scenarios: \nScenario-1: A pair of user action and app problem exists. For that write the output as follows: \n[{*insert app problem-1*: [*insert user action-1*]}]\nScenario-2: An app problem may have multiple user actions in sequence that lead to that app problem. For that, write the output as follows: \n[{*insert app problem-1*: [*insert user action-1*, *insert user action-2*]}] \nScenario 3: A user action can lead to multiple app problems. For that write the output as follows: \n[{*insert app problem-1*: [*insert user action-1*]}, {*insert app problem-2*: [*insert user action-1*]}] \nScenario-4: There are no app problems with corresponding user action, or there are app problems with no user actions. For that, we just keep it empty: \nJust write the format given above and nothing more. Do not write any other generated tests when I give an input. "}
]

In [9]:
import pandas as pd
df_1 = pd.read_csv("/content/Training_data.csv", header=None)

In [10]:
df_1


,0,1
0,"When I try to log in, it says my password is i...",[{'Incorrect password error': ['Try to log in']}]
1,The app crashes every time I open the settings.,[{'App crashes': ['Open the settings']}]
2,I can't upload my profile picture; it always f...,[{'Profile picture upload fails': ['Upload my ...
3,The app freezes when I try to send a message.,[{'App freezes': ['Send a message']}]
4,I get an error when I attempt to update my acc...,[{'Error updating account details': ['Update m...
...,...,...
364,The app's auto-fill feature doesn't work.,[]
365,The app's spell check is not accurate.,[]
366,The app's storage management is not effective.,[]
367,The app's battery saver mode doesn't work.,[]


In [13]:
user_prompt = df_1[0]
assistant_prompt = df_1[1]

print(user_prompt)

0      When I try to log in, it says my password is i...
1        The app crashes every time I open the settings.
2      I can't upload my profile picture; it always f...
3          The app freezes when I try to send a message.
4      I get an error when I attempt to update my acc...
                             ...                        
364            The app's auto-fill feature doesn't work.
365               The app's spell check is not accurate.
366       The app's storage management is not effective.
367           The app's battery saver mode doesn't work.
368       The app's notification badges are not visible.
Name: 0, Length: 369, dtype: object


In [122]:
for i in range(len(user_prompt)):
  messages.append({"role": "user", "content": user_prompt[i]})
  messages.append({"role": "assistant", "content": assistant_prompt[i]})

In [15]:
print(messages)

[{'role': 'system', 'content': 'Suppose you are an analyst who needs to take a user review of an application as input and extract all the app problems, and the corresponding user actions that the user takes to encounter the app problem. \nAn app problem is defined as a problem faced when interacting with an application. A user action is defined as the interactions with the application. \nShow the outputs in the form of python list of dictionaries.\nThe list will contain dictionaries, each dictionary will have app problem as the key and a list of user actions in sequential order as values. It is based on the following scenarios: \nScenario-1: A pair of user action and app problem exists. For that write the output as follows: \n[{*insert app problem-1*: [*insert user action-1*]}]\nScenario-2: An app problem may have multiple user actions in sequence that lead to that app problem. For that, write the output as follows: \n[{*insert app problem-1*: [*insert user action-1*, *insert user acti

In [58]:
response_arr = []
df_2 = pd.read_csv("/content/selected_reviews.csv", header=None)

In [59]:
df_2

,0
0,content
1,I had the app for long time but lately have st...
2,Too complicated to use
3,"Purchased by premium, getting error can not sy..."
4,It's takes a lot of space
...,...
196,"I used to like, even love this app. It was gre..."
197,For task duration you need premium
198,Lack of reminder in free version
199,Any event with repeat will not save. New updat...


In [61]:
array_required = df_2[0]

In [62]:
print(array_required)

0                                                content
1      I had the app for long time but lately have st...
2                                 Too complicated to use
3      Purchased by premium, getting error can not sy...
4                              It's takes a lot of space
                             ...                        
196    I used to like, even love this app. It was gre...
197                   For task duration you need premium
198                     Lack of reminder in free version
199    Any event with repeat will not save. New updat...
200                             It doesn't work at all!!
Name: 0, Length: 201, dtype: object


In [63]:
from openai import OpenAI
client = OpenAI(api_key = openai.api_key)

new_array = []
for i in range(190, 200):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 190
[{'Cannot manually rearrange "Important" list': ['Update the "Important" list feature']}]
ran 191
[]
ran 192
[]
ran 193
[{'Reminder does not function': ['Purchase premium version']}, {'No response from support team': ['Email support team', 'Request refund']}, {'Request refund': ['Request refund']}])
ran 194
[]
ran 195
[{'Reminders feature is premium': ['Use the reminders feature']}]
ran 196
[{'Annoying ads during adding events on calendar': ['Add an event on calendar']}]
ran 197
[]
ran 198
[]
ran 199
[]


In [65]:
for i in range(180, 190):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 180
LOL greedy app devs. They charge $36 annually or $48 if you select the monthly option. If you get the premium version, it'll allow you to receive push-notifications for a scheduled task. Like, to set a timer / alert for something on your list. Uninstalled. For $40, the app better be making you breakfast in the morning.
[]
ran 181
Used this app for months with no issues but after most recent update it plays an ad every time I add a new event to my calender, awful
[{'Plays ad for every new event': ['Add a new event to calendar']}]
ran 182
Hi have been looking for a app which remind me each and every hour because I'm a student and I need a that kind of app but in your application this feature is not available... 😒😒😔
[{'Feature request: Hourly reminders'}]
ran 183
No pomo time more than 1hr. No whitelist available. No feature for tracking task progress. Also add the feature to quit bad habits.
[{'No pomo time more than 1hr': []}, {'No whitelist available': []}, {'No feature for tra

In [66]:
for i in range(170, 180):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])

  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 170
Useless
[]
ran 171
I did not understand
[]
ran 172
I really love this app, as it really helps my focus on my work. But the recent update took away all of my motivation to collect coins to buy new species of trees. I'm not a premium user and not being able to collect all of the species makes me very sad.
[]
ran 173
Can't even create an account. I tried several times to open the app and start using it, but it says I don't have internet connection. My internet is fine, the app is not.
[{'No internet connection error': ['Open the app', 'Create an account']}]
ran 174
Sharing does not work in ANY way. For basic ToDo its OK, Sharing its Absolute TRASH. If you share a list with somebody and that somebody addicts and item or creates an item you don't see any changes. The person that you share a list with only sees an initial review of your list however no changes are propagated between the owner and sharing destination. And what is even worse is if the sharing destination user updates a

In [67]:
for i in range(160, 170):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 160
Use this app every day and used to totally love it. But in the last month or so they have allowed full-page video ads to take over the screen. The only way to get rid of them is to force close the app, which I now do repeatedly. Can't continue like this! Totally unacceptable. What the heck could they possibly have been thinking. Downright stupid business decision. Looking hard for another app to replace this one as soon as I can. It was nice while it lasted.
[{'Full-page video ads take over screen': ['Use app']}]
ran 161
Not so great app apart from its natural language input. Only recently added kanban view but lacks any basic requirements like reminders in the free version. Also the android widget hangs almost always. Never gets updated.
[{'Lacks reminders in free version': ['General feedback']}]
ran 162
Really liked the app and have been using premium but now I want to transfer to a new device. I can only doing this by signing up but whenever I go to the sign up section and t

In [68]:
for i in range(150, 160):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 150
not even a trial!
[]
ran 151
It's not the best option for beating distractions, the makers of this app are extremely money minded.
[]
ran 152
not very user friendly, atleast for me. the alarm doesnt ring it on only vibrates. i have to put every thing manually, like if its 11 am in the clock it gives me optionts what i m going to do from 12 pm. and even after using it for a week i am not clear how dis format gonna helm me
[]
ran 153
Only 5 habits. If you dont want to pay it's not worth having.
[]
ran 154
It is not giving me a reminder as a notification on my notification bar it is also not giving me any notification in the form of sound or vibration
[{"Not giving reminder notifications": ["Set reminder"]}]
ran 155
I have used this app for years but there is now a major issue. If you drag a task to tomorrow or place it under a task with a later date the task date no longer changes date. Is there any way to revert to an earlier revision? Hi I have sent screenshots but the issue is

In [69]:
for i in range(140, 150):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 140
Tried to put in my first event and it failed to log it. I uninstalled it same day.
[{'Failed to log event': ['Put in first event']}]
ran 141
Everything included in premium feature . Only frame work is there.
[]
ran 142
I don't care for the dashboard
[]
ran 143
Too complicated. It probably has a lot of, but it's not intuitive enough.
[]
ran 144
I DO NOT LIKE IT
[]
ran 145
Unable to share to do list with another account
[{'Unable to share to do list': ['Share to do list with another account']}]
ran 146
No good
[]
ran 147
It doesn't remind me. I even forgot I still had it on my phone.
[]
ran 148
U bought the app but can't reach my premium account
[]
ran 149
, every time I put a task in it removed the number in the name and put it in my inbox.... Why???
[{'Task number missing in name': ['Put task in inbox']}]


In [70]:
for i in range(130, 140):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 130
Too complicated, wanted to use to strategize for my exam but it was to difficult to add a task.
[]
ran 131
not working in my phone
[]
ran 132
doesnt really work for me personally, you might feel it differently however
[]
ran 133
Useless, ring bells only one time, and not a good reminder
[]
ran 134
requires microsoft account. deal breaker.
[]
ran 135
Edit: I'm using this application many year since release. And i'm loved it and donate, Now you make this application subscription $15 a year? Really?? Not happy with that...
[{'Not happy with subscription cost': ['Use the application', 'Donate']}], [{'Subscription cost issue': ['Use the application', 'Donate']}]
ran 136
This app is very, very, very sexist. When you start the app up and go into the "icon" menu for the avatar of your "hero" character, the vast majority of the icons possibile to select are definitively male.   Worse yet is that all of the ones 'intresting' in thier design - or portray something such as the workplace ar

In [71]:
for i in range(130, 140):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 130
Too complicated, wanted to use to strategize for my exam but it was to difficult to add a task.
[]
ran 131
not working in my phone
[]
ran 132
doesnt really work for me personally, you might feel it differently however
[]
ran 133
Useless, ring bells only one time, and not a good reminder
[]
ran 134
requires microsoft account. deal breaker.
[]
ran 135
Edit: I'm using this application many year since release. And i'm loved it and donate, Now you make this application subscription $15 a year? Really?? Not happy with that...
[{'Not happy with subscription pricing': ['Use the application']}]}
ran 136
This app is very, very, very sexist. When you start the app up and go into the "icon" menu for the avatar of your "hero" character, the vast majority of the icons possibile to select are definitively male.   Worse yet is that all of the ones 'intresting' in thier design - or portray something such as the workplace are entirely male.  The few female icons avalible - more than half not eve

In [73]:
for i in range(120, 130):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 120
You have to pay for all the decent features... There's nothing you want to do that your alarm doesn't already do
[]
ran 121
someone created a time productivity app that takes too much time out of your day to understand it.
[]
ran 122
Not at all satisfied about new update. Cannot unlock beautiful trees without pro version searching for similar apps and uninstall as soon as i find it
[{'Unsatisfied with new update': ['Unable to unlock trees without pro version', 'Looking for similar apps', 'Uninstall when finding similar app']}]
ran 123
It just don't work the way I thought it would. Not what I expected.
[]
ran 124
Good but hard to use
[]
ran 125
Since full screen ads were introduced, this calendar app has become very frustrating to use. Despite having found the app to be one of the best calendar apps available, the ads are now too much of a hindrance.
[{"Frustrating due to full screen ads": ['Use the calendar app']}]
ran 126
I already have subscription but still I'm not able to l

In [74]:
for i in range(110, 120):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 110
Logs me out without notice
[{'Logs me out without notice': []}]
ran 111
Got a new phone and app now won't allow me to hit sign in button. Customer service is very slow to respond to help. Love the app for last 2 / 3 yrs but not being able to use now is not worth my $$$ paid for it & to think it just renewed
[]
ran 112
Find good app. Buy good app. Copy some features from it. Ignore all others. New app doesn't do half of what the old app does. Kill the old app to make people use new app. People move to other third party app. 10/10 development strategy, well done Microsoft.
[]
ran 113
Garbage! Takes a simple concept and turns in to a spam generator.
[]
ran 114
Not as advertised
[]
ran 115
I don't know anything can anyone help me
[]
ran 116
Have to pay $4/mo if you want the reminders to work. I have ADD. The reminders are the #1 thing I need in a to-do list!
[{"Reminders not working": ["Sign up for reminders"]} ]
ran 117
What is going on? The app says "server connection lost"
[]
ra

In [75]:
for i in range(100, 110):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 100
For anyone who wants to use this app: Devs stopped updating this in 2018. So they won't reply to your queries and/or issues. Look elsewhere...
[]
ran 101
Dindigul
[]
ran 102
Boot it up, a choice of login or register. Chose uninstall.
[]
ran 103
Seems broken - cant complete the process of creating new tasks, so I'm just stuck with a huge empty menu I cant change. Loved the idea but this is broken and unusable.
[{"Can't complete process of creating tasks": ['Create new tasks']}]
ran 104
I don't liked becous it is not have the time in my tasks day 😭😧😩😢☹️🙄
[]
ran 105
Opaque, complicated, janky. Overall, a frustrating and confusing ap.
[]
ran 106
bruh I literally have no clue how this app works
[]
ran 107
I PAID FOR THIS YESTERDAY AND I GOT MY PREMIUM ONLY FOR ONE DAY??? TODAY I'M NOT PREMIUM AGAIN?? I had enough disappointment because my 2nd device didn't get premium after I PAID and now my first device also like this??? I emailed you since yesterday and also write on feedback on a

In [77]:
for i in range(90, 100):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 90
Was good. But not keeping up with the world. Having font size a pay feature with today's high resolution displays make the text so tiny that it can't be read. So in order to use the free version I have to pay for it.
[]
ran 91
Everything behind a pay wall...
[]
ran 92
$50 for a simple app? No thanks.
[]
ran 93
Don't bother unless you pay them($7.99yr) they don't offer a 
trial(suspicious) and you get NO access to features without paying. Too 
many other apps just like this that are much cheaper and let you TRY the 
pay version before paying.
[]
ran 94
This app was perfectly fine before Microsoft got its hands on it, I used it for years for pretty much everything, it was perfect! Now I cant share lists with people unless they are in my organization, lists don't sync, or just hang whenever I try to force it to sync, people don't receive my requests within a list. This app is now a complete waste of space on my phone, and after years of wanderlist I am now forced to try find a new 

In [78]:
for i in range(80, 90):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 80
How do I save changes to dailies(I have a ton of subtasks I curate as I hone in my routines) The save confirm button is gone! Also the app feels so different than the last version and it's a shock but that's par for the course with updates.
[{'Save button is missing': ['Curate subtasks', 'Save changes']}]
ran 81
Sends mails claiming monthly payments of $4 without yearly commitment but monthly without yearly commitment costs $7 And also the app is quite crappy without premium, and premium at $4 would still be too expensive for what you get.
[{'Misleading payment claim': ['Access premium features']}]
ran 82
App starts ads after 1 month and opens wanting paid every time after the first month.
[]
ran 83
The old navigation was better, especially on desktop. I miss being able to edit directly, now I have to go through more steps.
[]
ran 84
Hopeless
[]
ran 85
Can't print after last update
[{"Can't print": ['Update the app']}]
ran 86
Worst most useless app I've had the misfortune to try

In [79]:
for i in range(70, 80):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 70
The latest update now not working fine at all. It's worsening day by day. The developers now appears to have a different intention that premium paid version is purchased by the users.
[]
ran 71
This app used to be great when paired with Alexa. Every once in awhile the sync would break but disabling the app and relinking the account would fix the issue. This has not been the case for the past several months. Any.do it is no longer syncing with Alexa and based on the reviews of the Alexa app it is affecting a lot of people, has been ongoing for weeks, and there doesn't appear to be a fix coming. Going to give anylist a try to see if it works with Alexa.
[{'Sync issue with Alexa': ['Disable and relink account']}]
ran 72
It does not send me notification when i open this application it shows troubleshooting background running
[]
ran 73
Ads with free apps are acceptable, but fullscreen & non-skippable ads blocking the entire screen for 20 seconds are NOT. Awful user experience. Google

In [80]:
for i in range(60, 70):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 60
There is no a daily checklist where i can see what is remaining for today
[]
ran 61
Too many ads, find another alternatve with no ads or at least not as many.
[]
ran 62
Interface is not really satisfactory
[]
ran 63
Dear developers, thank you for deleting all my reminders after updating the app! It was not pleasant at all, believe me!
[{'Reminders deleted after update': ['Updating the app']}]
ran 64
App keeps crashing
[]
ran 65
This is worst app....single star dena majboori th...nhi to wo bh n deta....mera premium ke liye 129 rupees pay ho gya aur yha pe premium bh nhi mila....ye log bina login ke payment krate h...to kya proof h ki kisne payment ki.....this is worst app don't pay
[{'Worst app review': ['Express dissatisfaction with premium purchase']}]
ran 66
I want to love this app, but every time I use it and every time I try to upgrade to premium, I get an "Error 599" message. Tried factory resetting my phone, even! Definitely a shame, as I was looking for an app to recommen

In [81]:
for i in range(50, 60):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 50
Just give an option to go back to the old version. So unsatisfied and feel demotivated after the new update.
[]
ran 51
Its not even opening 😠😠😡😠😡😠😡😠😠 this don't deserve even one star. This is just a wastage of time and data .
[]
ran 52
Phone is 6 months old. This app along with Outlook, Office and Teams really drained the battery
[{'App drained battery': ['Use the app', 'Use Outlook', 'Use Office', 'Use Teams']}]
ran 53
Programmer does nog work properly. It is changing times and dates on his own. Instead of saving time it is costing me time (and money for the pro version). I ran into this problem a few days after my purchase so i'm not able to get a refund while the app is unreliable, so practically unusuable.
[]
ran 54
Oohh im very hardly managing it and its a disgusting app just a waste of time. Also it is not uninstalling.
[]
ran 55
I don't use the free version, I like to pay for software that deserves it. I tapped on the app today and had an interstitial ad. Do not do this a

In [82]:
for i in range(40, 50):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 40
Very useful app! The only thing I wish you put on the app is a pause button or something to point different moments in the work block. Excelent app I bought the pro version 'cause is very accesible. I wish you put more things to make this app the most incredible productivity app! 🤍 EDIT: I need help with the premium purchase!!! I send an email and one person responded my email but the problem was not solved. I want a refund please, I like the app but this problem is a Headache, sorry. :/
[]
ran 41
Paid for premium. Sync does not work. Publisher does not reply. Started reading reviews and found numerous people with the same problem. Going to try to get my money back. Nice app if you don't care about backups or syncing between devices or customer support. Switched to Goalify and finding them extremely responsive
[]
ran 42
This is an overengineered list keeping app that I hate. I kept it for years in spite of that because I was in the ecosystem, but today it locked me out of my gro

In [83]:
for i in range(30, 40):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 30
while using the habbit feature, you can add multiple reminders for drinking water, but if you check done in any one of those reminders then it wont remind for the rest of the day. ruins the logic. Uninstalled Installed a year later and still the issue exists, the habit feature was a unique point which brought me to the app but I guess I'll have to use another one.
[{'Habit feature issue': ['Use habit feature']}]
ran 31
La pubblicità è diventata insopportabile
[]
ran 32
I don't really liked it much . Waste of intenet.
[]
ran 33
I'd love to use this app if there was any other payment method than with a credit card (which I don't have and don't want).
[{}]
ran 34
Very worst as compared to Google calendar. I think Google calendar and your this apps to be put it in the litter bins.
[{}]
ran 35
It doesn't work anymore:(
[]
ran 36
not able to login..say not login with personal account..I have only one microsoft account
[{'Not able to login': ['Login with personal account']}]
ran 37
I g

In [84]:
for i in range(20, 30):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 20
Doesn't look like the pictures at all. Exact same layout as your phone's calendar. Now new settings etc. Waste of time app.
[]
ran 21
Find a new app. Overall if you havent noticed, the dev's left this app. As of recently, the servers are dead. This means you cant make an account, forums are gone, motivation pictures gone, etc. Find a new app,
[]
ran 22
Used to be a good app, now it's all buggy...
[]
ran 23
I got the premium version The main drawbacks of this app are: 1- I cannot customize the categories 2- If I do a task earlier, it still alarming on scheduled alarming time 3- removing done tasks is not available Just moved to the bottom
[{"Cannot customize categories": []}, {"Task alarming time issue": []}, {"Removing done tasks not available": []}]
ran 24
Way to confusing to use the tutorial didnt help at all
[{}]
ran 25
no user friendly, i download this apk for only n only set my full day 
sedual and follow up it everyday but i do not do easy way
[]
ran 26
Video ads
[]
ran 27

In [85]:
for i in range(10, 20):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 10
Poor service. I purchased the premium version, but I can't login again. When I choose the login function, the app always asks for redemption and has no function of filling in the account login.
[{'Can't login with premium account': ['Choose the login function']}]
ran 11
Dont use this app for real!!!
[]
ran 12
It's completely broken, things don't show up, error codes from the very beginning, it's just broken
[]
ran 13
All functionality comes with premium account only... Shud have more flexibility for those who are looking for free version
[]
ran 14
I am colour blind and dyslexic. Not able to read anything. I can't zoom in and I need to pay in order to see the colours better. [update] As I said, I would have to pay the app in order to have the right fonts and colour so I can actually read and make use with it. I've tried WeNote, and it's the app I need, without having to pay premium.
[{'Cannot read due to color blindness and dyslexia': ['Use the app']}]
ran 15
Unfortunately i can 

In [86]:
for i in range(0, 10):
  updated_message = messages + [{"role": "user", "content": array_required[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(array_required[i])
  print(response.choices[0].message.content.strip())
  new_array.append(response)

ran 0
content
I'm sorry, but I will need specific input related to app problems and user actions to generate the desired output. Thank you.
ran 1
I had the app for long time but lately have started has the longest and most annoying advs that you can't even close your phone without watching them all. Plenty of other apps so no reason to keep it, unistall immediately.
[]
ran 2
Too complicated to use
[]
ran 3
Purchased by premium, getting error can not sync. Emailed support, no response. Don't waste your time or money
[{'App sync error': ['Purchase premium']}]
ran 4
It's takes a lot of space
[]
ran 5
After installing the app, i check to see if my reacurring event would show up. And it did that one time. So minutes after i insalled i had to let it go.
[]
ran 6
This app is really pretty annoying. it said I didn't allow any permissions but it gave absolutely no further instruction. So for a $1.99 I figured what do I have to lose. Well I have a lot of time to lose because I spent the money I 

In [98]:
df_new = pd.read_csv("/content/scenario1.csv")

In [99]:
df_new

,Reviews,Outputs
0,I try to upload a photo befre the app crashes.,{'the app crashes': ['try to upload a photo']}
1,I log in untl the app freezes.,{'the app freezes': ['log in']}
2,I open the settings menu thn the app stops res...,{'the app stops responding': ['open the settin...
3,I click on a notification bfre the app shuts d...,{'the app shuts down': ['click on a notificati...
4,I start a video call ten the app lags.,{'the app lags': ['start a video call']}
5,I send a message bfr the app crashes.,{'the app crashes': ['send a message']}
6,I switch to another tab untl the app becomes u...,{'the app becomes unresponsive': ['switch to a...
7,I attempt to open a file befor the app crashes.,{'the app crashes': ['attempt to open a file']}
8,I try to access my profile util the app freezes.,{'the app freezes': ['try to access my profile']}
9,I navigate to the chat section thn the app sto...,{'the app stops responding': ['navigate to the...


In [100]:

scenario_1_prompt = list(df_new["Reviews"])

In [103]:
scenario_1_prompt

['I try to upload a photo befre the app crashes.',
 'I log in untl the app freezes.',
 'I open the settings menu thn the app stops responding.',
 'I click on a notification bfre the app shuts down.',
 'I start a video call ten the app lags.',
 'I send a message bfr the app crashes.',
 'I switch to another tab untl the app becomes unresponsive.',
 'I attempt to open a file befor the app crashes.',
 'I try to access my profile util the app freezes.',
 'I navigate to the chat section thn the app stops responding.',
 'I tap on a notification bfre the app shuts down.',
 'I start a call thn the app lags.',
 'I write a comment bfore the app crashes.',
 'I switch to a different view util the app becomes unresponsive.',
 'I try to upload a document befre the app crashes.',
 'I login util the app freezes.',
 'Evrytime I try to log in, it says my session expired and kicks me out.',
 "The app keeps updating constantly, evn though I've already updated it.",
 "I can't log into my Facebook account no

In [104]:
array_for_scenario_2 = []
for i in range(0, 10):
  updated_message = messages + [{"role": "user", "content": scenario_1_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_1_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_2.append(response)

ran 0
I try to upload a photo befre the app crashes.
[{'app crashes': ['upload a photo']}]
ran 1
I log in untl the app freezes.
[{App freezes: [Log in]}]
ran 2
I open the settings menu thn the app stops responding.
[{'App stops responding': ['Open settings menu']}]
ran 3
I click on a notification bfre the app shuts down.
[{'App shuts down': ['Click on a notification']}]
ran 4
I start a video call ten the app lags.
[{'App lags': ['Start a video call']}]
ran 5
I send a message bfr the app crashes.
[{App crashes: [Send a message]}]
ran 6
I switch to another tab untl the app becomes unresponsive.
[{'App becomes unresponsive': ['Switch to another tab']}]
ran 7
I attempt to open a file befor the app crashes.
[{'App Crashes': ['Open file']}]
ran 8
I try to access my profile util the app freezes.
[{"App freezes": ["Access profile"]}]
ran 9
I navigate to the chat section thn the app stops responding.
[{"App stops responding": ["Navigate to the chat section"]}]


In [106]:
array_for_scenario_2 = []
for i in range(10, 20):
  updated_message = messages + [{"role": "user", "content": scenario_1_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_1_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_2.append(response)

ran 10
I tap on a notification bfre the app shuts down.
[{"App shuts down": ["Tap on a notification"]}]
ran 11
I start a call thn the app lags.
[{'App lags': ['Start a call']}]
ran 12
I write a comment bfore the app crashes.
[{App Crash: [Write a comment before the app crashes]}]
ran 13
I switch to a different view util the app becomes unresponsive.
[{'App becomes unresponsive': ['Switch to a different view']}]
ran 14
I try to upload a document befre the app crashes.
[{App crashes: [Upload document]}]
ran 15
I login util the app freezes.
[{'App freezes': ['Login']}]
ran 16
Evrytime I try to log in, it says my session expired and kicks me out.
[{"Session expired": ["Log in"]}]
ran 17
The app keeps updating constantly, evn though I've already updated it.
[{The app keeps updating constantly: [I've already updated it]}]
ran 18
I can't log into my Facebook account no matter how many times I try.
[{"unable to log into Facebook": ["attempted to log in multiple times"]}]
ran 19
Facebook's algo

In [107]:
array_for_scenario_2 = []
for i in range(20, 30):
  updated_message = messages + [{"role": "user", "content": scenario_1_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_1_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_2.append(response)

ran 20
The app logs me out randomly and makes me go through two-factor authentication evrtim I try to log back in.
[{'Random log out': ['Attempt to log in', 'Go through two-factor authentication']}]
ran 21
Suddenly, the app won't show anything on my page aftr I log in .
[{"App not showing anything after log in": []}]
ran 22
My Facebook completely crashed, and now I can't log in at all.
[{Facebook crashed: [Unable to log in]}]
ran 23
After the latest update, the app won't let me log in or create a new account.
[{login issue: [cannot log in]}, {create account issue: [cannot create a new account]}]
ran 24
My account got hacked, and now I can't get any help from customer support.
[{Account Hacked: [Contact Customer Support]}]
ran 25
The auto-refresh on the news feed makes it impossible to read articles or comments properly.
[{"Auto-refresh on news feed": ["Disable auto-refresh feature"]}]
ran 26
I can't block all reels, and the ones I can block keep coming back. 
[{"I can't block all reels

In [108]:
array_for_scenario_2 = []
for i in range(30,40):
  updated_message = messages + [{"role": "user", "content": scenario_1_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_1_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_2.append(response)

ran 30
It won't log me in and keeps saying my session expired whenver I try to log in.
[{'session expired error': ['try to log in']}]
ran 31
The app repeatedly tells me to try again whnevr I try to log in, even after resetting my password.
[{'app repeatedly tells me to try again whenever I try to log in': ['try to log in', 'reset password']}]
ran 32
App constantly crashing when trying to post a status
[{App constantly crashing: [trying to post a status]}]
ran 33
Notifications not showing up when scrolling through the feed
[{Notifications not showing up when scrolling through the feed: [Scrolling through the feed]}]
ran 34
Videos freezing randomly when watching videos
[{Videos freezing randomly: [watching videos]}]
ran 35
Messages not sending when trying to send a message
[{Messages not sending: [trying to send a message]}]
ran 36
Photos not uploading properly when uploading a photo
[{Photos not uploading properly: [uploading a photo]}]
ran 37
Settings not saving changes when accessing 

In [109]:
array_for_scenario_2 = []
for i in range(40, len(scenario_1_prompt)):
  updated_message = messages + [{"role": "user", "content": scenario_1_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_1_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_2.append(response)

ran 40
Profile information not updating whn updating profile information
[{Profile information not updating: [Update profile information]}]
ran 41
Reactions not registering wen reacting to a post
[{Reactions not registering wen reacting to a post: [Check internet connection, Try refreshing the page]}]
ran 42
Stories not loading whn viewing stories
[{Stories not loading: [Viewing stories]}]
ran 43
Likes not showing up whn liking a post
[{Likes not showing up: [Liking a post]}]
ran 44
Shares not going through wen trying to share a post
[{'Shares not going through': ['Trying to share a post']}]
ran 45
Events not displaying whe accessing events
[{Events not displaying: [Accessing events]}]
ran 46
Groups not accessible whn trying to join a group
[{'Groups not accessible': ['Trying to join a group']}]
ran 47
Pages not loading properly wen accessing pages
[{Pages not loading properly: [Accessing pages]}]
ran 48
Search results not relevant hen searching
[{Search results not relevant: [Searchin

In [110]:
df_3 = pd.read_csv("/content/scenario3.csv")

In [113]:
scenario_3_prompt = list(df_3["Reviews"])

In [114]:
scenario_3_prompt

["Everytime I open the app, it crashes unexpectedly. Can't even get started! This crash happens right after I attempt to open the camera to capture some photos.",
 "As soon as I enter my login details, the app crashes. Can't access my account! This crash consistently occurs right after inputting my username and password.",
 "Before the latest update, notifications were working fine. Now they've disappeared completely! This issue started happening immediately after I updated the app.",
 "When I try to tap on any button, there's no response from the app. Can't navigate properly! This unresponsive behavior occurs whenever I try to interact with any button in the app.",
 'While using GPS in the maps app, it freezes intermittently. Makes it difficult to get directions! This freezing occurs everytime I try to use the GPS feature while navigating.',
 'Everytime I refresh the feed, images fail to load. Missing out on updates! This problem persists whenever I try to refresh the feed to view new

In [125]:
array_for_scenario_3 = []
for i in range(40, len(scenario_3_prompt)):
  updated_message = messages + [{"role": "user", "content": scenario_3_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_3_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_2.append(response)

ran 40
Battery drains quickly everytime I use the app. Need to recharge constantly! This rapid battery drain happens whenever I'm using the app.
[{"Battery drains quickly": ["Use the app"]}]
ran 41
While watching a video, sound keeps cutting out randomly. Can't enjoy any audio content! This sound cutting issue occurs everytime I try to watch a video.
[{"Sound cutting out during video playback": ["Watch a video"]}]
ran 42
As soon as I try to access settings, the menu abruptly closes. Can't adjust any preferences! This problem happens consistently whenever I try to access settings.
[{'Abrupt menu closing': ['Access settings']}]
ran 43
Before the recent update, fonts were clear and easy to read. Now they look distorted! This issue started occurring after I updated the app.
[{"Fonts distorted": ["Update the app"]}]
ran 44
Everytime I open the app, it switches to random screens. Confusing and disruptive! This screen switching issue happens every time I open the app.
[{'Random screen switchi

In [129]:
array_for_scenario_2 = []
for i in range(30, 40):
  updated_message = messages + [{"role": "user", "content": scenario_3_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_3_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_2.append(response)

ran 30
After trying to use the camera feature, I realized it's inaccessible within the app. Can't take any photos! This problem happens whenever I try to use the camera.
[{"Can't take photos": ['Use the camera']}]
ran 31
Before I can even start using the app, it crashes unexpectedly. Extremely frustrating! This crash occurs immediately after launching the app.
[{'App crashes on launch': ['Launch the app']}]
ran 32
As soon as I click on the login button, the app freezes. Can't access my account! This freeze occurs right after attempting to log in.
[{'Freeze after login attempt': ['Click on the login button']}]
ran 33
After inputting my login details, the app crashes. Stuck on the login screen! This crash happens consistently after entering my username and password.
[{'App crashes on login screen': ['Enter username and password']}]
ran 34
Before the latest update, notifications used to work fine. Now they've disappeared completely! This issue started happening after I updated the app.
[{

In [ ]:
array_for_scenario_2 = []
for i in range(20, 30):
  updated_message = messages + [{"role": "user", "content": scenario_3_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_3_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_2.append(response)

In [127]:
array_for_scenario_2 = []
for i in range(10, 20):
  updated_message = messages + [{"role": "user", "content": scenario_3_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_3_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_2.append(response)

ran 10
As soon as I try to access settings, the menu abruptly closes. Can't adjust any preferences! This problem happens consistently whenever I try to access the settings menu.
[{'Settings menu abruptly closes': ['Access settings menu']}]
ran 11
Before the recent update, fonts were clear and easy to read. Now they look distorted! This issue started occurring immediately after I updated the app.
[{'Distorted fonts': ['Update the app']}]
ran 12
Everytime I open the app, it switches to random screens. Confusing and disruptive! This screen switching issue happens every time I open the app.
[{'Random screen switching': ['Open the app']}]
ran 13
While trying to sync data, it doesn't appear on my other devices. Syncing issue needs attention! This syncing problem occurs whenever I try to sync data between devices.
[{'Syncing issue needs attention': ['Sync data between devices']}]
ran 14
After trying to use the camera feature, I realized it's inaccessible within the app. Can't take any photos!

In [128]:
array_for_scenario_2 = []
for i in range(0, 10):
  updated_message = messages + [{"role": "user", "content": scenario_3_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_3_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_2.append(response)

ran 0
Everytime I open the app, it crashes unexpectedly. Can't even get started! This crash happens right after I attempt to open the camera to capture some photos.
[{'App crashes': ['Open the camera']}]
ran 1
As soon as I enter my login details, the app crashes. Can't access my account! This crash consistently occurs right after inputting my username and password.
[{'App crashes': ['Enter login details', 'Log in']}]
ran 2
Before the latest update, notifications were working fine. Now they've disappeared completely! This issue started happening immediately after I updated the app.
[{'Notifications disappeared': ['Update the app']}]
ran 3
When I try to tap on any button, there's no response from the app. Can't navigate properly! This unresponsive behavior occurs whenever I try to interact with any button in the app.
[{'Unresponsive behavior with buttons': ['Interact with any button']}]
ran 4
While using GPS in the maps app, it freezes intermittently. Makes it difficult to get directions

In [132]:
df_2 = pd.read_csv("/content/scenario2_1.csv")

In [133]:
df_2

,Reviews,Outputs
0,This app is infuriating. I tried to transfer m...,[{'app crashes': ['transfer money to a friend'...
1,Terrible experience trying to pay my utility b...,[{'app freezes': ['enter payment details for u...
2,"I tried booking a flight and hotel package, an...",[{'app times out': ['search for flight and hot...
3,Trying to book a rental car was a nightmare. T...,"[{'app is slow': ['select rental car']}, {'car..."
4,"I tried to log my workout, and it was a frustr...",[{'app takes forever to load': ['log workout']...
5,Logging meals on this app is a joke. The barco...,[{'barcode scanner rarely works': ['log meals'...
6,"Terrible app. I tried to purchase a few items,...",[{'app removes items from cart': ['add items t...
7,Horrible user experience trying to order a gif...,"[{'app is slow': ['add items to cart']}, {'app..."
8,Terrible service. The internet is constantly d...,[{'internet constantly drops': ['use internet'...
9,Very disappointed with FastNet. The connection...,[{'connection is unreliable': ['use internet']...


In [134]:
scenario_2_prompt = df_2["Reviews"]

In [135]:
array_for_scenario_2 = []
for i in range(40, len(scenario_2_prompt)):
  updated_message = messages + [{"role": "user", "content": scenario_2_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_2_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_2.append(response)

ran 40
Trying to use this app to schedule a home repair was a nightmare. It was slow to load available appointment slots, and when I finally found one, the app crashed during confirmation. After restarting, my booking was lost, and I had to repeat the process. To make matters worse, customer support was unresponsive when I tried to get assistance.
[{'Slow to load available appointment slots': ['Schedule a home repair']}, {'App crashes during confirmation': ['Schedule a home repair']}, {'Booking lost after app restart': ['Schedule a home repair']}, {'Unresponsive customer support': ['Schedule a home repair']}]
ran 41
I attempted to use this app to order a birthday cake, but it was a disaster. The app was sluggish when browsing cake designs, and it frequently froze during checkout. After finally completing the order, the app crashed, and my payment was not processed. Despite multiple attempts to contact customer service, I received no response.
[{'Sluggish browsing for cake designs': ['O

In [136]:
#3
array_for_scenario_2 = []
for i in range(30, 40):
  updated_message = messages + [{"role": "user", "content": scenario_2_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_2_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_2.append(response)

ran 30
Using this app to order food was a complete disaster. The app was slow to load the menu items, and when I finally selected my order, it crashed during checkout. After reopening the app, my cart was empty, and I had to start over. Once I managed to place the order, the delivery was late, and some items were missing. Customer service was unresponsive, and I never got a refund for the missing items.
[{'App slow to load menu items': ['Order food']}, {'App crashes during checkout': ['Order food']}, {'Cart is empty after app reopens': ['Order food']}, {'Delivery is late': ['Order food']}, {'Items missing from order': ['Order food']}, {'Unresponsive customer service': ['Order food']}, {'No refund for missing items': ['Order food']}]
ran 31
I tried to use this app to apply for a loan, and it was a frustrating experience. The app kept giving me errors during the application process, and when I finally completed it, there was no confirmation. I tried checking my application status, but th

In [137]:
#3
array_for_scenario_2 = []
for i in range(20, 30):
  updated_message = messages + [{"role": "user", "content": scenario_2_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_2_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_2.append(response)

ran 20
Attempting to reset my password was a nightmare. The app constantly gave me an 'error' message when I tried to submit my new password. When it finally accepted a password, the confirmation email never arrived. I tried logging in with the new password, but it didn't work. Customer support was useless and kept telling me to try the same steps over and over.
[{'Password reset nightmare': ['Try to reset password']}]
ran 21
[{'Slow loading of library and crashes': ['Download music']}, {'Songs not playing offline': ['Download music']}, {'Redownloading songs multiple times': ['Download music']}, {'Songs not working properly': ['Download music']}]
ran 22
I tried to use the app to order groceries, and it was a complete mess. The app was slow and unresponsive while browsing items. Several items in my cart were suddenly marked as unavailable at checkout, even though they were listed as in stock. After finally placing the order, the delivery status kept showing errors, and I never received 

In [138]:
#
array_for_scenario_2 = []
for i in range(10, 20):
  updated_message = messages + [{"role": "user", "content": scenario_2_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_2_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_2.append(response)

ran 10
I tried to sign up for a new account and the app was a mess. It kept saying my email was invalid even though it was correct. When I finally got past that, the verification code never arrived in my inbox. After requesting it multiple times, I received several codes at once, none of which worked. Customer service didn't help at all. Frustrating experience.
[{"Invalid email error": ["Sign up"]}, {"Verification code not received": ["Sign up"]}, {"Multiple codes received": ["Sign up"]}, {"Customer service not helpful": ["Sign up"]}]
ran 11
Booking a concert ticket through this app was a disaster. First, the app crashed multiple times while searching for events. When it finally loaded, the seat selection was glitchy and wouldn't let me choose my preferred seats. After several tries, I managed to book a ticket, but then the app double-charged my card. Now I have two charges and only one ticket. Unacceptable.
[{"App crashes during search for events": ["Search for events"]}, {"Glitchy se

In [139]:
array_for_scenario_2 = []
for i in range(0, 10):
  updated_message = messages + [{"role": "user", "content": scenario_2_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_2_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_2.append(response)

ran 0
This app is infuriating. I tried to transfer money to my friend, but the process was a nightmare. First, the app crashed multiple times while trying to access my contacts. When it finally worked, the app couldn't recognize my friend's username despite it being correct. After several attempts, the transfer finally went through, but then I noticed that the app had deducted the amount twice from my account! I contacted customer service, but they were unresponsive. Avoid this app at all costs.
[{'App crashes accessing contacts': ['Transfer money to friend']}, {'App doesn't recognize friend username': ['Transfer money to friend']}, {"Transaction deducted amount twice": ['Transfer money to friend']}, {"Customer service unresponsive": ['Contact customer service']}]
ran 1
Terrible experience trying to pay my utility bill using this app. The app froze while entering my payment details and then crashed completely. After restarting, I had to re-enter all my information, only to get an error

In [140]:
df_3 = pd.read_csv("/content/scenario3_1.csv")

In [141]:
scenario_3_prompt = df_3["Reviews"]

In [142]:
#2
array_for_scenario_3 = []
for i in range(40, len(scenario_3_prompt)):
  updated_message = messages + [{"role": "user", "content": scenario_3_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_3_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_3.append(response)

ran 40
Tried to upload a photo on Instagram but encountered an issue with the filters. Selected a photo from the gallery and tried to apply a filter, but the filter options didn't appear. Closed and reopened the app, but the filters were still missing. Restarted the phone and tried again, but the filters remained unavailable.
[{'Filter options not appearing': ['Select photo', 'Apply filter']}, {'Filter options remain missing': ['Close and reopen app', 'Restart phone']}]
ran 41
Attempted to send a message on Facebook Messenger but encountered a glitch. Typed out the message and clicked on the send button, but the message failed to deliver. Refreshed the chat and tried again, but encountered the same issue. Checked for app updates, installed the latest version, and attempted to send the message again, but still no success.
[{"Message failed to deliver": ['Type out the message', 'Click send button', 'Refresh chat']}]
ran 42
Posted a tweet on Twitter but noticed a problem with the characte

In [144]:
#2
array_for_scenario_3 = []
for i in range(30, 40):
  updated_message = messages + [{"role": "user", "content": scenario_3_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_3_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_3.append(response)

ran 30
Tried to watch a video on TikTok but encountered an issue with the audio. Adjusted the volume settings but couldn't hear anything. Closed and reopened the app, but the audio problem persisted.
[{'Audio issue': ['Watch a video on TikTok', 'Adjust volume settings', 'Close and reopen the app']}]
ran 31
Attempted to send a direct message on Instagram but encountered a glitch. Typed out the message and attached a photo, but the send button was unresponsive. Restarted the app and tried again, but the glitch persisted.
[{'Direct message glitch': ['Type out message', 'Attach a photo', 'Attempt to send message']}]
ran 32
Posted a status update on Facebook but noticed a formatting issue. Typed out the status and added a link, but the formatting appeared distorted. Deleted and retyped the status, but encountered the same issue.
[{'Formatting issue on Facebook status update': ['Type out the status and add a link', 'Delete and retype the status']}]
ran 33
Clicked on a hashtag on Twitter but 

In [145]:
#4
array_for_scenario_3 = []
for i in range(20, 30):
  updated_message = messages + [{"role": "user", "content": scenario_3_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_3_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_3.append(response)

ran 20
Tried to access my LinkedIn account but couldn't log in. Entered the correct email and password multiple times, but the login screen kept reappearing. Reset my password but still couldn't access the account.
[{'Unable to log in to LinkedIn account': ['Enter email and password', 'Reset password']}]
ran 21
Encountered an issue while watching reels on Instagram. Tried to pause a video using the pause button, but the video continued playing. Closed and reopened the app, but the problem persisted.
[{'Issue with pause button': ['Watch reels on Instagram']}]
ran 22
Posted a photo on Facebook and attempted to add a caption. Typed out the caption but received an error message when trying to post. Refreshed the app and tried again, but encountered the same error.
[{'Error posting caption': ['Type out caption', 'Post photo']}]
ran 23
Received a notification on Instagram but couldn't view the full content. Tapped on the notification but was redirected to the app home screen. Checked my noti

In [146]:
#5
array_for_scenario_3 = []
for i in range(10, 20):
  updated_message = messages + [{"role": "user", "content": scenario_3_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_3_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_3.append(response)

ran 10
Encountered an issue while trying to post a photo on my Instagram story. Added a filter to the photo, but received an error message. Attempted to add text to the photo, but encountered the same error. Finally, tried tagging someone in the story, but the error persisted.
[{'Error posting Instagram story': ['Add filter to photo', 'Add text to photo', 'Tag someone in story']}]
ran 11
Discovered a bug on LinkedIn where my profile appeared duplicated. Looked for a solution within the app settings, but couldn't find any relevant options. Reported the issue to LinkedIn support and waited for a response.
[]
ran 12
While watching reels on Instagram, used the seek bar to skip ahead in a video. However, the seek bar suddenly disappeared, and the video froze. Tried refreshing the app, but the issue persisted.
[{'Seek bar disappears and video freezes': ['Skip ahead in a video']}, {'Issue persists after refreshing the app': ['Refresh the app']}]
ran 13
Experienced an accessibility issue on Fa

In [147]:
#5
array_for_scenario_3 = []
for i in range(0, 10):
  updated_message = messages + [{"role": "user", "content": scenario_3_prompt[i]}]
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = updated_message

  )
  print("ran " + str(i))
  print(scenario_3_prompt[i])
  print(response.choices[0].message.content.strip())
  array_for_scenario_3.append(response)

ran 0
I tried to update my profile picture, but every time I attempted to upload a new photo, I received an error message. Thinking it might be a temporary glitch, I refreshed the app and tried again, but unfortunately, the same error persisted. Even after checking my internet connection and ensuring it was stable, I couldn't get the photo to upload. As a last resort, I logged out and logged back in, hoping it would resolve the issue, but to no avail.
[{'Error uploading profile picture': ['Refresh the app', 'Upload a new photo']}, {'Error uploading profile picture': ['Log out', 'Log in']}]
ran 1
Recently, I encountered a frustrating issue while trying to send a message to a friend. Despite multiple attempts, the message failed to deliver each time. I refreshed the app, thinking it might help, but the problem persisted. I even restarted my phone and reinstalled the app, but unfortunately, I still couldn't send messages. As a workaround, I had to resort to using a different messaging pla